In [1]:
#|default_exp fire_detection

# Fire detection

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import geopandas as gpd
import ee

#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee
import geemap
from datetime import datetime

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils
from gee_projects import const_vars as const
from gee_projects import mask_s2 as mask_s2
from gee_projects import img_sen2 as img_sen2

In [4]:
# access image using the img_sen2 class
extractor = img_sen2.access_s2_image(path_to_geometry=r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg", 
                            crs = "4326",
                            start_date = '2023-10-06',
                            end_date = '2023-10-08',
                            one_day_image = True,
                            user_date = '2023-10-07')

img=extractor.img

Requested date: 2023-10-07
The nearest date available: 2023-10-07


Calculate indices

In [5]:
def BAI(img):
    bai_img=img.expression(
        '(1-((B6*B7*B8A)/B4)**0.5)*((B12-B8A)/((B12+B8A)**0.5)+1)', {
                'B6': img.select('B6'),
                'B7': img.select('B7'),
                'B8A': img.select('B8A'),
                'B4': img.select('B4'),
                'B12': img.select('B12')})

    return bai_img

In [6]:
img_BAI = BAI(img).rename('BAI')
img_BAI.bandNames().getInfo()

['BAI']

In [9]:
Map = geemap.Map()

# Visualization parameters
visualization_bai = {
    'bands': ['BAI'],
    'min': -1,
    'max': 7,
    'opacity': 0.5,
    'palette': [
      '477eff','2872ff','2414ff','f5ff68','fffa28',
      'ffe30a','ffaf18','ff6a20','ff0402','ea2aff'
    ]
}

visualization_rgb = {'bands': ['B4','B3','B2'],
    'min': 0,
    'max': 1,
    'opacity': 1,
}

Map.add_basemap("SATELLITE")

Map.addLayer(extractor.img.select(['B4','B3','B2']), visualization_rgb, "img1")
Map.addLayer(img_BAI,visualization_bai,'BAI')
Map.centerObject(extractor.fc)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…